In [1]:
import pandas as pd 
import os

In [2]:
geoDF = pd.read_csv('./geodata.csv', index_col = 0)

In [3]:
geoDF.head()

,Site Name,Location,Coordinates,Landscape,Climate,Reference,Site PI,Data Available
Site ID,,,,,,,,
1,Bermuda Island,"Bermuda Island, England","32°15’N, 64°52’W",Coast,Dwa,Steen-Larsen et al. (2015),Hans Christian Steen-Larsen,In prep.
2,Beijing,"Beijing, China","40°0’N, 116°3’E",Urban,Dwa,Wen et al. (2010),Xuhui Lee and Xuefa Wen,Yes
3,Borden,"Borden, Canada","44°9′N, 79°6′W",forest,Dfb,Santos et al. (2012),Xuhui Lee,Yes
4,Camargue,"Vaccarès lagoon, France","43°35’N, 4°28’E",Wetland,Csb,Delattre et al. (2015),Christine Vallet-Coulomb,Yes
5,Dome C,"Concordia, South Pole","75°06′S, 123°23′E",Ice Sheet,-,Casado et al (2016),Mathieu Casado,Yes


In [10]:
isodfs = []
isopath = "./isotope-data/"
for filename in os.listdir(isopath):
    print(isopath + filename)
    df = pd.read_csv(isopath + filename)
    if 'Delta_18O' in df.columns && 'Delta_D' in df.columns:
        df = df[df['Delta_18O'] != -9999 && df['Delta_D']!= -9999]
        if(len(df.index) > 0):
            print(filename + " has nontrivial data")
            isodfs.append(df)
    

./isotope-data/JP-Nagoya.csv
./isotope-data/Ship-RARAAVIS.csv
Ship-RARAAVIS.csv has nontrivial data
./isotope-data/SP-Teide.csv
SP-Teide.csv has nontrivial data
./isotope-data/Ship-PIRATAFR24.csv
Ship-PIRATAFR24.csv has nontrivial data
./isotope-data/US-NewHaven.csv
US-NewHaven.csv has nontrivial data
./isotope-data/CN-Gaoqiao.csv
CN-Gaoqiao.csv has nontrivial data
./isotope-data/AP-Kohnen.csv
AP-Kohnen.csv has nontrivial data
./isotope-data/CH-Rietholzbach.csv
CH-Rietholzbach.csv has nontrivial data
./isotope-data/AP-DomeC.csv
AP-DomeC.csv has nontrivial data
./isotope-data/Ship-JARE55.csv
Ship-JARE55.csv has nontrivial data
./isotope-data/US-Rosemount_G21.csv
US-Rosemount_G21.csv has nontrivial data
./isotope-data/Ship-Bermuda.csv
Ship-Bermuda.csv has nontrivial data
./isotope-data/CN-Beijing.csv
CN-Beijing.csv has nontrivial data
./isotope-data/Ship-ACTIV.csv
Ship-ACTIV.csv has nontrivial data
./isotope-data/RU-Kourovka.csv
RU-Kourovka.csv has nontrivial data
./isotope-data/US-Manit

In [16]:
isodfs[3]

,TIMESTAMP_START,TIMESTAMP_END,Q,Delta_18O,Delta_18O_std,Delta_D,Delta_D_std,T,RH,P,...,Rn,U,WD,T_ERA5,RH_ERA5,P_ERA5,Preciptition_ERA5,Rn_ERA5,U_ERA5,WD_ERA5
19,200312032300,200312032359,-9999.000,-34.756300,-9999.000000,-9999.000000,-9999.000000,-3.3,0.40,103.250975,...,-9999.00000,2.572000,320,-9999.00000,-9999.000000,-9999.00000,-9999.0,-9999.00000,-9999.000000,-9999.0000
20,200312040000,200312040059,-9999.000,-34.154000,-9999.000000,-9999.000000,-9999.000000,-3.3,0.44,103.250975,...,-9999.00000,2.572000,-9999,-9999.00000,-9999.000000,-9999.00000,-9999.0,-9999.00000,-9999.000000,-9999.0000
21,200312040100,200312040159,-9999.000,-31.897700,-9999.000000,-9999.000000,-9999.000000,-6.7,0.62,103.217111,...,-9999.00000,2.057600,60,-9999.00000,-9999.000000,-9999.00000,-9999.0,-9999.00000,-9999.000000,-9999.0000
22,200312040200,200312040259,-9999.000,-30.664000,-9999.000000,-9999.000000,-9999.000000,-7.8,0.77,103.183247,...,-9999.00000,0.000000,0,-9999.00000,-9999.000000,-9999.00000,-9999.0,-9999.00000,-9999.000000,-9999.0000
23,200312040300,200312040359,-9999.000,-28.262800,-9999.000000,-9999.000000,-9999.000000,-3.9,0.50,103.183247,...,-9999.00000,1.543200,-9999,-9999.00000,-9999.000000,-9999.00000,-9999.0,-9999.00000,-9999.000000,-9999.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19580,200805312300,200805312359,16140.182,-12.348370,0.686665,-101.917010,19.997632,19.0,0.73,97.911000,...,136.89200,4.444444,150,18.59016,0.896827,99.80355,0.0,116.05460,6.002643,215.8927
19581,200806010000,200806010059,16334.498,-12.711785,1.206765,-116.074530,27.944969,18.9,0.84,97.911000,...,106.26000,4.444444,180,18.41288,0.900197,99.92703,0.0,18.85889,4.194046,247.6961
19582,200806010100,200806010159,14721.815,-12.161473,0.700791,-99.994028,19.447139,16.7,0.87,98.010000,...,206.58800,0.000000,0,18.00855,0.909255,99.96425,0.0,-30.18278,3.908118,244.2020
19583,200806010200,200806010259,14686.496,-12.413896,0.843791,-93.973065,14.995728,17.2,0.81,98.010000,...,119.37200,2.666667,220,17.51061,0.913192,99.95241,0.0,-19.25548,4.637161,226.6708
